In [1]:
# install detectron2 if want to recognize different types of pedestrians
# installation instructions for M1 Mac: https://medium.com/@hakon.hukkelas/installing-detectron2-natively-for-mac-m1-pro-apple-silicon-a89517f1c913
#!pip install git+https://github.com/facebookresearch/detectron2@main
# add cfg.MODEL.DEVICE = 'cpu' to your configuration.

In [3]:
from models import *
from utils import *

import numpy as np
import os, sys, time, datetime, random
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torchvision

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import cv2
import pandas as pd
import utm
from tqdm.notebook import tqdm
from IPython.display import clear_output
from matplotlib.animation import FuncAnimation
from celluloid import Camera


# Use SORT to track people
from sort import *

# setup detectron2
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.video_visualizer import VideoVisualizer
from detectron2.utils.visualizer import ColorMode, Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
print("torch version", torch.__version__)

ModuleNotFoundError: No module named 'sort'

# Configs

In [3]:
# To be changed 
video_filename = 'Y2E2_West.MOV' # put video in './data' directory
detector = 'detectron2' # yolov3, faster_rcnn, detectron2

# precomputed homography matrices
############# STADIUM #############
# pts_img = [[450.0,203.0],[282.0,297.0],[470.0,403.0],[630.0,270.0]] # top left, bot left, bot right, top right 
# pts_world = [[250,250], [250,500], [500,500],[500,250]]

# HOMOG, status = cv2.findHomography(np.array(pts_img), np.array(pts_world)+700)

############# GCS #############
# HOMOG = [[4.97412897e-02, -4.24730883e-02, 7.25543911e+01],
#          [1.45017874e-01, -3.35678711e-03, 7.97920970e+00],
#          [1.36068797e-03, -4.98339188e-05, 1.00000000e+00]] 

############# Y2E2 West #############
#note: points must be consistent in order and winding (e.g. clockwise) 
pts_img = [[1090, 1625], [2270, 1785], [2010, 2250], [470, 2150]] # palm tree locations 
long_lat_world = [(37.428570, -122.175184), (37.428489, -122.175211), 
                  (37.428506, -122.175288), (37.428589, -122.175260)]
pts_world = []
for lat,lon in long_lat_world: 
    utm_east, utm_north, utm_zone, utm_letter = utm.from_latlon(lat,lon) # meters
    pts_world.append([utm_east*100, utm_north*100]) # centimeters
# shift pts_world so that values aren't too big
world_np = np.array(pts_world)
world_np = world_np - world_np.min(axis=0)
pts_world = world_np.tolist()

    
HOMOG, status = cv2.findHomography(np.array(pts_img), np.array(pts_world))

In [4]:
# Usually don't need to change
assumed_height = 174 # cm 
history_size = 5
speed_tolerance = 20

yolov3_weight_path = 'weights/yolov3.weights'
yolov3_config_path='config/yolov3.cfg'

current_time = datetime.datetime.now().strftime("%m%d%y_%H%M%S")
path_result = os.path.join('results', detector, video_filename.split('.')[0], current_time)
os.makedirs(path_result, exist_ok=True)
result_path_detection = os.path.join(path_result, video_filename)
result_path_world = result_path_detection.replace('.', '_2D.',1)

scale_size = None
conf_thres=None
nms_thres=None

if detector=='yolov3':
    scale_size = 416
    conf_thres=0.8
    nms_thres=0.4
    CLASSES = utils.load_classes('config/coco.names') # 80 classes for darknet pretraiend
elif detector=='faster_rcnn': 
    # 91 classes torchvision pretrained
    # from https://pytorch.org/vision/stable/models.html#id57
    CLASSES = [ 
        '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
        'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
        'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
        'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A',
        'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
        'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
        'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
        'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
        'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table',
        'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
        'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book',
        'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
    ]
elif detector=='detectron2': 
    # segmentation
    CLASSES = utils.load_classes('config/coco.names') # 80 classes for detectron2 pretraiend
    cfg = get_cfg()
    if not torch.cuda.is_available():
        cfg.MODEL.DEVICE = 'cpu' 
    
    #### Coco segmentation ####
    # add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
    cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
    # Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
    
    #### Coco keypoint ####
#     cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml"))
#     cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7  # set threshold for this model
#     cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml")
    
    #### Coco panoptic segmentation #### TODO
#     cfg.merge_from_file(model_zoo.get_config_file("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml"))
#     cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7  # set threshold for this model
#     cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml")
else: 
    raise Exception('Invalid detector name')
    

# Inference Only - Panoptic Segmentation w/ Detectron2 

In [5]:
#### converting any coordinate to world coordinate 
def img_to_world_coords(coords_img, HOMOG, pts_img = None, pts_world = None):
    '''
    use at least 4 
    input: 
        coords_img: a np array of dim (num_ped,seq_len,2)
    if using pre-computed HOMOG, no need to use pts_img and pts_world
        pts_img: a list of [x,y] points in image coordinates. list length cannot be shorter than 4
        pts_world: a list of [x,y] points in world coordinates. each point cooresponds to a point in pts_img
        assumed_height: the estimated height of all humans. source: https://www.sciencedirect.com/science/article/pii/S0379073811002167
                        NOT USED FOR NOW 
    output: 
        coords_world: same shape as coords_img but in world coordinates
    '''
    # convert to np matrix and check size
    if pts_img == None: 
        h = HOMOG
    else: 
        pts_img = np.array(pts_img)
        pts_world = np.array(pts_world)
        assert pts_img.shape==pts_world.shape and pts_img.shape[0] >= 4
     
        # calculate homography matrix H. more pts_img means more accurate H
        h, status = cv2.findHomography(pts_img, pts_world)
     
    # finally, get the mapped world coordinates
    coords_world = cv2.perspectiveTransform(coords_img.astype(np.float32), h)
    # coords_world[:,1] += assumed_height
    return coords_world

def calc_num_violated (all_pedestrians):
    num_violated = 0
    
#     for p in all_pedestrians: 
#         min_y = 100000
#         if p.y2 < min_y: 
#             min_y = p.y2
#             closest_p = p

#     radius_cm = closest_p.box_w * assumed_height / closest_p.box_h / 2
#     worlddist2cm = radius_cm / closest_p.radius 
#     print(worlddist2cm)
    from itertools import combinations 

    for (p1,p2) in combinations(all_pedestrians, 2): 
        if p1.is_violating_sd(p2):
            assert p2.is_violating_sd(p1)
            p1.is_violating=True
            p2.is_violating=True
    
    for p in all_pedestrians: 
        if p.is_violating:
            num_violated += 1
            
    return num_violated

def draw_circle(frame, center, radius, color=None): 
    cv2.circle(frame, (int(center[0]), int(center[1])), int(radius), color=color, thickness=10)

def draw_point(frame, center): 
    draw_circle(frame, center, 1, color=(0,255,0))

def detect_image(frame, model, detector_name, scale_size = None):
    '''
    input: a SINGLE opencv (BGR) image, usually just a frame from cv2.videocapture.read()
    returns: 
    predictions - a prediction dictionary, containing output in 2 formats:
                      prediction["darknet"] = FloatTensor[N,6], each row is x1, y1, x2, y2, class_score, class_pred
                      prediction["detectron2"] = original detectron2 output.
                      prediction["torchvision"] = torchvision format. 
    '''
    prediction = dict()
    
    # make sure model is correct mode
    if detector is not 'detectron2':
        model.to(device=device)
        model.eval()
    
    # scaling and padding (optional)
    rgb_img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    if scale_size: 
        pilimg = Image.fromarray(rgb_img) # pillow image 
        img_size = scale_size
        ratio = min(img_size/pilimg.size[0], img_size/pilimg.size[1])
        imw = round(pilimg.size[0] * ratio)
        imh = round(pilimg.size[1] * ratio)
        img_transforms = transforms.Compose([ transforms.Resize((imh, imw)),
             transforms.Pad((max(int((imh-imw)/2),0), max(int((imw-imh)/2),0), max(int((imh-imw)/2),0), max(int((imw-imh)/2),0)),
                            (128,128,128)),
             transforms.ToTensor(),
             ])
        # convert image to Tensor
        rgb_tensor = img_transforms(pilimg).float()
        
    else: 
        # convert rgb image to Tensor
        rgb_tensor = np.moveaxis(rgb_img, -1, 0) / 255 # convert to (C,H,W) and [0-1]
        rgb_tensor = torch.tensor(rgb_tensor, device=device).float()
    
    ######## torchvision model detection ########
    '''
    Getting predictions, torchvision style (https://pytorch.org/vision/stable/models.html#id57)
    - input: list of tensors, one for each image. Tensor shape [C,H,W], values within [0,1]
    - return: List[Dict[Tensor]], one dict for each image
        "boxes": FloatTensor[N,4] in [x1,y1,x2,y2] format
        "labels": Int64Tensor[N] 
        "scores": Tensor[N]
    - We here convert predictions to be consistent with darknet format:
        FloatTensor[N,7]
        each row is x1, y1, x2, y2, class_score, class_pred
    '''
    if detector_name == 'faster_rcnn':
        # torchvision wants RGB input
        with torch.no_grad():
            prediction["torchvision"] = model([rgb_tensor]) # takes input as a list of tensor
        for pred_dict in prediction["torchvision"]: 
            boxes = pred_dict['boxes'] # (N,4)
            classIDs = pred_dict['labels'] # (N)
            scores = pred_dict['scores'] # (N)    
            
            num_boxes = boxes.shape[0]
            prediction["darknet"] = torch.cat((boxes, scores.unsqueeze(1), classIDs.unsqueeze(1)),dim=1)
    
    ######## YOLOV3 model detection ########
    elif detector_name == 'yolov3': 
        '''
        - Input: [num_image,C,H,W]
        - Darknet returns: [centerx, centery, width, height, score, classA prob, ... classB prob]
          utils.non_max_suppresion converts results to list of num_image tensors:FloatTensor[N,7]
              each row is x1, y1, x2, y2, object_conf, class_score, class_pred
        '''
        # darknet wants RGB input
        with torch.no_grad():
            rgb_tensor = torch.unsqueeze(rgb_tensor, dim=0)
            predictions = model(rgb_tensor) 
            print(predictions)
        predictions = utils.non_max_suppression(predictions, num_classes=len(CLASSES), conf_thres=conf_thres, nms_thres=nms_thres)
        prediction["darknet"] = predictions[0] # [0] because 1 image only. 
        # remove column containing object_conf. 
        prediction["darknet"] = torch.cat([prediction["darknet"], prediction["darknet"]],dim=1)
        
    ######## detectron2 panoptic segmentation ########
    elif detector_name == 'detectron2': 
        '''
        - Input: cv2 image 
        - Return: 
        '''
        # detectron2 wants opencv BGR input, so dont need to use RGB image. 
        prediction["detectron2"] = model(frame) # output format: https://detectron2.readthedocs.io/en/latest/tutorials/models.html#model-output-format
        # convert to darknet format
        N = len(prediction["detectron2"]["instances"].pred_boxes)
        prediction["darknet"] = torch.FloatTensor(N, 6).to(device) 
        prediction["darknet"][:,0:4] = prediction["detectron2"]["instances"].pred_boxes.tensor # first 4 columns: x1, y1, x2, y2
        prediction["darknet"][:,4] = prediction["detectron2"]["instances"].scores
        prediction["darknet"][:,5] = prediction["detectron2"]["instances"].pred_classes
    else: 
        raise Exception('Invalid detector name')
    return prediction

##### custom pedestrian class #####
class Pedestrian:
    def __init__(self, coords1, coords2, pedestrian_id, cls, box_h, box_w):
        self.x1, self.y1 = coords1 # top lef
        self.x2, self.y2 = coords2 # bot right 
        self.id = pedestrian_id # in this frame 
        self.cls = cls # either 'person' or 'bicycle'
        self.box_h = box_h
        self.box_w = box_w

        feet = img_to_world_coords(np.array([[[self.x1, self.y2],[self.x2,self.y2]]]), HOMOG).squeeze()
        feetL = feet[0,:]
        feetR = feet[1,:]
        
        self.radius = np.linalg.norm(feetL - feetR)/2 
        
#         center_x =(feetL[0] + feetR[0])/2
#         center_y = (feetL[1] + feetR[1])/2
        self.center = (feetL + feetR)/2 # np.array([center_x, center_y])
        self.history = []
        self.history_timestamps = []
        self.speed = 0
        self.is_static = False
        self.is_active=True
        self.is_violating = False
        
        
    def is_violating_sd(self, ped2, threshold): # 182.88 cm = 6 ft. threshold = 100 cm according to WHO  
        '''
        return true if self and ped2 are too close
        '''
        distance = np.linalg.norm(ped2.center - self.center) - ped2.radius - self.radius
#         distance *= worlddist2cm 
        if distance < threshold: 
            return True
        else: 
            return False 
        
##### TODO: not used / tested yet ####
class PedestrianManager: 
    def __init__(self, FPS, history_size=5, speed_tolerance = 10, threshold=5): 
        self.FPS = FPS
        self.history_size = history_size
        self.speed_tolerance = speed_tolerance
        self.threshold = threshold
        self.pedestrians_dict = dict() # stores all pedestrians including those that are no longer active 
        self.current_time = 0
        self.active_ids = set() # set of all pedestrian IDs that are still active in the video 
        
    def __len__(self): 
        return len(self.pedestrians_dict)
    
    def __iter__(self):
        return iter(self.pedestrians_dict)
    
    def __contains__(self, pedestrian_id): 
        if pedestrian_id in self.pedestrians_dict.keys(): 
            return True
        else: 
            return False
        
    def __getitem__(self, pedestrian_id): 
        return self.pedestrians_dict[pedestrian_id]
    
    def __setitem__(self, pedestrian_id, p): 
        history = []
        history_timestamps = []
        if pedestrian_id in self: 
            # not a new pedestrian, get old history
            history = self[pedestrian_id].history
            history_timestamps = self[pedestrian_id].history_timestamps
            
        # initialize a new active pedestrian with updated history 
        history.append(p.center)
        history_timestamps.append(self.current_time)
        
        # remove oldest history
        if len(history) > self.history_size: 
            history.pop(0)
            history_timestamps.pop(0)
            
        p.history = history
        p.history_timestamps = history_timestamps
        
        # calculate current speed of the pedestrian 
        if len(history) > 1:
            p.speed = np.linalg.norm(history[-1] - history[-2]) / (p.history_timestamps[-1] - p.history_timestamps[-2]) # distance / time
        else: 
            p.speed = 0
            
        if p.speed <= self.speed_tolerance: 
            p.is_static = True
            
        # store it in our container class  
        self.active_ids.add(pedestrian_id)
        self.pedestrians_dict[pedestrian_id] = p
        
    def __set_inactive(self, pedestrian_id):
        if pedestrian_id in self.active_ids:
            self.active_ids
        else:
            raise Exception("Pedestrian {} is not currently active".format(pedestrian_id))
            
    def advance_time(self): 
        # set as inactive if no data in last frame 
        new_set = set()
        for k in self.active_ids: 
            if self[k].history_timestamps[-1] != self.current_time:
                # inactive
                self[k].is_active = False
            else:
                # active 
                new_set.add(k)
        self.active_ids = new_set
        # advance time by 1 frame
        self.current_time += 1 / self.FPS 
        
    def calc_num_violated(self):
        from itertools import combinations 
        
        violating = set()
        for (id1,id2) in combinations(self.active_ids, 2): 
            p1 = self[id1]
            p2 = self[id2]
            if (not p1.is_static) and (not p2.is_static) and p1.is_violating_sd(p2, self.threshold):
                assert p2.is_violating_sd(p1, self.threshold)
                p1.is_violating=True
                p2.is_violating=True
                violating.add(p1.id)
                violating.add(p2.id)
        
        num_violated = len(violating)
        return num_violated

In [6]:
cmap = plt.get_cmap('tab20b')
colors = [cmap(i)[:3] for i in np.linspace(0, 1, 20)]

# initialize model and detectron2 visualizer 
if detector == 'faster_rcnn':
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
elif detector == 'yolov3': 
    model = Darknet(yolov3_config_path, img_size=scale_size) #torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True) #
    model.load_weights(yolov3_weight_path)
elif detector == 'detectron2': 
    model = DefaultPredictor(cfg)
    detectron2_visualizer = VideoVisualizer(MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), ColorMode.IMAGE)
else: 
    raise Exception('Invalid detector name')
    
# initialize object tracker
mot_tracker = Sort() 

In [7]:
# load video
cap = cv2.VideoCapture(os.path.join('data', video_filename))
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
FPS = int(cap.get(cv2.CAP_PROP_FPS))
num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
result1 = cv2.VideoWriter(result_path_detection, 
                cv2.VideoWriter_fourcc(*'MJPG'),
                FPS, 
                (width, height))
fig, ax = plt.subplots(figsize=(10,10))
camera = Camera(fig)
# container class 
all_pedestrians = PedestrianManager(FPS, history_size=history_size, speed_tolerance=speed_tolerance)

# while cap.isOpened():
for ii in tqdm(range(700)): # just do this many frames for now 
    ret, frame = cap.read()
    
    if ret is False:
        break
        
    prediction = detect_image(frame, model, detector_name = detector, scale_size = scale_size)
    
    if prediction is None: 
        break
        
    # plot boxes on frame and show inline
    pilimg = Image.fromarray(frame)
    img = np.array(pilimg)
    if scale_size:
        # need to resize box to account for how image was padded to square
        pad_x = max(img.shape[0] - img.shape[1], 0) * (scale_size / max(img.shape))
        pad_y = max(img.shape[1] - img.shape[0], 0) * (scale_size / max(img.shape))
        unpad_h = scale_size - pad_y
        unpad_w = scale_size - pad_x
    else: 
        # no padding.
        pad_x = 0 
        pad_y = 0
        unpad_h = img.shape[0]
        unpad_w = img.shape[1]

    # draw segmentation mask if detectron
    if detector == 'detectron2': 
        if "panoptic_seg" in prediction["detectron2"].keys():
            panoptic_seg, segments_info = prediction["detectron2"]["panoptic_seg"]
            out = detectron2_visualizer.draw_panoptic_seg_predictions(frame, panoptic_seg.to("cpu"), segments_info)
        else: 
            out = detectron2_visualizer.draw_instance_predictions(frame, prediction["detectron2"]["instances"].to("cpu"))
        frame = out.get_image() # BGR

    # track and visualize for regular detection boxes and ids. 
    tracked_objects = mot_tracker.update(prediction["darknet"].cpu())
    unique_labels = prediction["darknet"][:, -1].cpu().unique()
    n_cls_preds = len(unique_labels)
    for x1, y1, x2, y2, obj_id, cls_pred in tracked_objects:
        obj_id = int(obj_id)
        box_h = int(((y2 - y1) / unpad_h) * img.shape[0])
        box_w = int(((x2 - x1) / unpad_w) * img.shape[1])
        y1 = int(((y1 - pad_y // 2) / unpad_h) * img.shape[0])
        x1 = int(((x1 - pad_x // 2) / unpad_w) * img.shape[1])

        color = colors[obj_id % len(colors)]
        color = [i * 255 for i in color]
        cls = CLASSES[int(cls_pred)]
        if cls == 'person' : # 0: person
            cv2.rectangle(frame, (x1, y1), (x1+box_w, y1+box_h), color, 1)
            cv2.putText(frame, cls + "-" + str(int(obj_id)), (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 3)
            all_pedestrians[obj_id] = Pedestrian((x1, y1), (x1+box_w, y1+box_h), int(obj_id), cls, box_h, box_w)
    
    # see if anyone's too close? 
    num_proximity = all_pedestrians.calc_num_violated()
    cv2.putText(frame, "Number of People: "+str(len(all_pedestrians.active_ids))+"; Proximity alert: "+str(num_proximity), (10,frame.shape[0]-5), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,0,255), 3)
    
    # show 2D transformed world coordinates
    if ii == 0: 
        # initiate points so that we get a good axis proportion
        ax.scatter(*np.vstack([all_pedestrians[i].center for i in all_pedestrians.active_ids]).T, color='k', s=0.0001)
        ax.set_aspect('equal')
        
    for pedestrian_id in all_pedestrians.active_ids: 
        p = all_pedestrians[pedestrian_id]
        #  red color if violated 
        if p.is_violating: 
            color=(1.0,0.0,0.0)
            ax.annotate('colliding', p.center)
        else: 
            color = colors[int(p.id) % len(colors)] 
            # convert from cv2 BGR color to RGB
            color = (color[2], color[1], color[0])
        if p.is_static: 
            ax.annotate('static', p.center)
#         ax.annotate(p.id, p.center)
        circle = plt.Circle(p.center, p.radius, color=color)
        ax.add_patch(circle)
        
#     break
    #######save video to results folder###############
    result1.write(frame)
    camera.snap()
    #################################################
    
    # always call this before looking at the next frame 
    all_pedestrians.advance_time()

anim = camera.animate(blit=True) # save matplotlib animation with celloid
anim.save(result_path_world, fps=FPS)

cap.release()
result1.release()

  0%|          | 0/700 [00:00<?, ?it/s]

In [ ]:
anim = camera.animate(blit=True) # save matplotlib animation with celloid
anim.save(result_path_world, fps=FPS)

cap.release()
result1.release()
# result2.release()

In [ ]:
# (for debug only)
# check homography matrix 
%matplotlib inline
pts_img = [[1090, 1625], [2270, 1785], [2010, 2250], [470, 2150]] # palm tree locations 
print(pts_img)
print(pts_world)

flong_lat_world = [(37.428570, -122.175184), (37.428489, -122.175211), 
                  (37.428506, -122.175288), (37.428589, -122.175260)]
pts_world = []
for lat,lon in long_lat_world: 
    utm_east, utm_north, utm_zone, utm_letter = utm.from_latlon(lat,lon) # meters
    pts_world.append([utm_east*100, utm_north*100]) # centimeters
# shift pts_world so that values aren't too big
world_np = np.array(pts_world)
world_np = world_np - world_np.min(axis=0)
pts_world = world_np.tolist()


HOMOG, status = cv2.findHomography(np.array(pts_img), np.array(pts_world))


f, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(18,18))
pts_test_img = [[100, 100], [200, 300], [1000, 1000], [2000, 3000]]
pts_test_world = img_to_world_coords(np.array([pts_test_img]), HOMOG)[0,:,:].T
print(pts_test_world.shape)
ax2.scatter(*pts_test_world)
ax2.set_aspect('equal')
# ax1.imshow(cv2.cvtColor(warped_frame, cv2.COLOR_BGR2RGB))
# ax2.imshow(warped_frame)
ax3.imshow(frame)
ax3.scatter(*np.array(pts_test_img).T)
for i, (ptx, pty) in enumerate(pts_img): 
    ax3.scatter(ptx, pty)
    ax3.annotate(i, (ptx, pty), color='r')

# Archive 

In [ ]:
# load video (old, no container / manager class) 
cap = cv2.VideoCapture(os.path.join('data', video_filename))
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
FPS = int(cap.get(cv2.CAP_PROP_FPS))
num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
result1 = cv2.VideoWriter(result_path_detection, 
                cv2.VideoWriter_fourcc(*'MJPG'),
                FPS, 
                (width, height))
# result2 = cv2.VideoWriter(result_path_world, 
#                 cv2.VideoWriter_fourcc(*'MJPG'),
#                 FPS, 
#                 (1296,1296)) # matplotlib figsize (18,18) -> 1296
fig, ax = plt.subplots(figsize=(18,18))
camera = Camera(fig)

# while cap.isOpened():
for ii in tqdm(range(1000)): # just do this many frames for now 
    ret, frame = cap.read()
    
    if ret is False:
        break
        
    prediction = detect_image(frame, model, detector_name = detector, scale_size = scale_size)
    
    if prediction is None: 
        break
        
    # plot boxes on frame and show inline
    pilimg = Image.fromarray(frame)
    img = np.array(pilimg)
    if scale_size:
        # need to resize box to account for how image was padded to square
        pad_x = max(img.shape[0] - img.shape[1], 0) * (scale_size / max(img.shape))
        pad_y = max(img.shape[1] - img.shape[0], 0) * (scale_size / max(img.shape))
        unpad_h = scale_size - pad_y
        unpad_w = scale_size - pad_x
    else: 
        # no padding.
        pad_x = 0 
        pad_y = 0
        unpad_h = img.shape[0]
        unpad_w = img.shape[1]

    # draw segmentation mask if detectron
    if detector == 'detectron2': 
        if "panoptic_seg" in prediction["detectron2"].keys():
            panoptic_seg, segments_info = prediction["detectron2"]["panoptic_seg"]
            out = detectron2_visualizer.draw_panoptic_seg_predictions(frame, panoptic_seg.to("cpu"), segments_info)
        else: 
            out = detectron2_visualizer.draw_instance_predictions(frame, prediction["detectron2"]["instances"].to("cpu"))
        frame = out.get_image() # BGR

    # track and visualize for regular detection boxes and ids. 
    tracked_objects = mot_tracker.update(prediction["darknet"].cpu())
    
    all_pedestrians = []
    unique_labels = prediction["darknet"][:, -1].cpu().unique()
    n_cls_preds = len(unique_labels)
    for x1, y1, x2, y2, obj_id, cls_pred in tracked_objects:
        box_h = int(((y2 - y1) / unpad_h) * img.shape[0])
        box_w = int(((x2 - x1) / unpad_w) * img.shape[1])
        y1 = int(((y1 - pad_y // 2) / unpad_h) * img.shape[0])
        x1 = int(((x1 - pad_x // 2) / unpad_w) * img.shape[1])

        color = colors[int(obj_id) % len(colors)]
        color = [i * 255 for i in color]
        cls = CLASSES[int(cls_pred)]
        if cls == 'person' : # 0: person
            cv2.rectangle(frame, (x1, y1), (x1+box_w, y1+box_h), color, 1)
            cv2.putText(frame, cls + "-" + str(int(obj_id)), (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 3)
            all_pedestrians.append(Pedestrian((x1, y1), (x1+box_w, y1+box_h), int(obj_id), cls, box_h, box_w))
    
    # see if anyone's too close? 
    num_proximity = calc_num_violated (all_pedestrians)
    cv2.putText(frame, 
                "Number of People: "+str(len(all_pedestrians))+"; Proximity alert: "+str(num_proximity), 
                (10,frame.shape[0]-5), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                2, 
                (0,0,255), 
                3)
    
    # show 2D transformed world coordinates
    if ii == 0: 
        # initiate points so that we get a good axis proportion
        ax.scatter(*np.vstack([p.center for p in all_pedestrians]).T, color='k', s=0.0001)
        ax.set_aspect('equal')
        
    for p in all_pedestrians: 
        #  red color if violated 
        if p.is_violating: 
            p.plot_color=(1.0,0.0,0.0)
        else: 
            p.plot_color = colors[int(p.id) % len(colors)]

    #         ax.annotate(p.id, p.center)
        circle = plt.Circle(p.center, p.radius, color=p.plot_color)
        ax.add_patch(circle)
        
#     break
    #######save video to results folder###############
    result1.write(frame)
    camera.snap()
#     result2.write(warped_frame)
    #################################################

anim = camera.animate(blit=True) # save matplotlib animation with celloid
anim.save(result_path_world, fps=FPS)

cap.release()
result1.release()
# result2.release()

In [ ]:
# Initialize video capture
vid = cv2.VideoCapture(videopath)
width  = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(vid.get(cv2.CAP_PROP_FPS))
num_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

# initialize video writer 
result = cv2.VideoWriter(resultpath_detectron2, 
                cv2.VideoWriter_fourcc(*'MJPG'),
                fps, 
                (width, height))

# initialize video visualizer
v = VideoVisualizer(MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), ColorMode.IMAGE)

# while(True):
for ii in tqdm(range(600)): # just do this many number of frames for now 
    ret, frame = vid.read()
    outputs = predictor(frame)
    out = v.draw_instance_predictions(frame, outputs["instances"].to("cpu"))
    im = out.get_image()
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)

    # Write to video file
    result.write(im)

#     # (for debugging) Show output frame
#     plt.imshow(im)
#     break

vid.release()
result.release()

# Panoptic segmentation w/ detectron2

## test with screenshot

In [ ]:
im = cv2.imread('data/test_screenshot.png')
plt.imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))

In [ ]:
cfg = get_cfg()
cfg.MODEL.DEVICE = 'cpu' # Added for CPU only inference
# add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
# Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)
outputs = predictor(im)

In [ ]:
# look at the outputs. See https://detectron2.readthedocs.io/tutorials/models.html#model-output-format for specification
print(outputs["instances"].pred_classes)
print(outputs["instances"].pred_boxes)

# We can use `Visualizer` to draw the predictions on the image.
v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
im = out.get_image()[:, :, ::-1]
plt.imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))

In [ ]:
fig = plt.figure(figsize=(50,50))
plt.imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))

## now test it with video

In [ ]:
# Initialize video capture
vid = cv2.VideoCapture('data/Y2E2_West.MOV')
width  = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(vid.get(cv2.CAP_PROP_FPS))
num_frames = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))

# initialize video writer 
result = cv2.VideoWriter('results/Y2E2_West.mp4', 
                cv2.VideoWriter_fourcc(*'MJPG'),
                fps, 
                (width, height))

# initialize video visualizer
v = VideoVisualizer(MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), ColorMode.IMAGE)

# while(True):
for ii in tqdm(range(600)): # just do this many number of frames for now 
    ret, frame = vid.read()
    outputs = predictor(frame)
    out = v.draw_instance_predictions(frame, outputs["instances"].to("cpu"))
    im = out.get_image()

    # Write to video file
    result.write(im)

#     # (for debugging) Show output frame
#     plt.imshow(im)
#     break

vid.release()
result.release()

# YOLOv3 detection

In [ ]:
config_path='config/yolov3.cfg'
weights_path='weights/yolov3.weights'
class_path='config/coco.names'
img_size=416
conf_thres=0.8
nms_thres=0.4

# Load model and weights
model = Darknet(config_path, img_size=img_size)
model.load_weights(weights_path)
model.to(device)
model.eval()
classes = utils.load_classes(class_path)

In [ ]:
# Video specific magic numbers
assumed_height = 174
pts_img = [[1090, 1575], [2270, 1725], [470, 2050],[2010, 2150] ] # palm tree locations 
long_lat_world = [(37.428570, -122.175184), (37.428489, -122.175211), 
                  (37.428589, -122.175260), (37.428506, -122.175288)]
pts_world = []
for lat,lon in long_lat_world: 
    utm_east, utm_north, utm_zone, utm_letter = utm.from_latlon(lat,lon) # meters
    pts_world.append([utm_east*100, utm_north*100]) # centimeters
# shift pts_world so that values aren't too big
world_np = np.array(pts_world)
world_np = world_np - world_np.min(axis=0)
pts_world = world_np.tolist()

    
h, status = cv2.findHomography(np.array(pts_img), np.array(pts_world))

#### converting any coordinate to world coordinate 
def img_to_world_coords(coords_img, pts_img, pts_world):
    '''
    use at least 4 
    input: 
        coords_img: a np array of dim (num_ped,seq_len,2)
        pts_img: a list of [x,y] points in image coordinates. list length cannot be shorter than 4
        pts_world: a list of [x,y] points in world coordinates. each point cooresponds to a point in pts_img
        assumed_height: the estimated height of all humans. source: https://www.sciencedirect.com/science/article/pii/S0379073811002167
                        NOT USED FOR NOW 
    output: 
        coords_world: same shape as coords_img but in world coordinates
    '''
    # convert to np matrix and check size

    pts_img = np.array(pts_img)
    pts_world = np.array(pts_world)
    assert pts_img.shape==pts_world.shape and pts_img.shape[0] >= 4
     
    # calculate homography matrix H. more pts_img means more accurate H
    h, status = cv2.findHomography(pts_img, pts_world)
     
    # finally, get the mapped world coordinates
    coords_world = cv2.perspectiveTransform(coords_img.astype(np.float32), h)
    # coords_world[:,1] += assumed_height
    return coords_world

In [ ]:
# %pylab inline 
# from IPython.display import clear_output

cmap = plt.get_cmap('tab20b')
colors = [cmap(i)[:3] for i in np.linspace(0, 1, 20)]
# for pd dataframe
raw_data_list = []
csv_columns = ["frame_id", "agent_id", "pos_x", "pos_y"]

# initialize Sort object
mot_tracker = Sort() 

result1 = cv2.VideoWriter(resultpath_detection, 
                cv2.VideoWriter_fourcc(*'MJPG'),
                fps, 
                (width, height))
result2 = cv2.VideoWriter(resultpath_world, 
                cv2.VideoWriter_fourcc(*'MJPG'),
                fps, 
                (width, height))
# while(True):
for ii in tqdm(range(600)): # just do this many number of frames for now 
    ret, frame = vid.read()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    pilimg = Image.fromarray(frame)
    detections = detect_image(pilimg)

    img = np.array(pilimg)
    pad_x = max(img.shape[0] - img.shape[1], 0) * (img_size / max(img.shape))
    pad_y = max(img.shape[1] - img.shape[0], 0) * (img_size / max(img.shape))
    unpad_h = img_size - pad_y
    unpad_w = img_size - pad_x
    ########### vivian initialize ############
    all_pedestrians = [] # all of the people in this frame 
    ##########################################
    if detections is not None:
        tracked_objects = mot_tracker.update(detections.cpu())

        unique_labels = detections[:, -1].cpu().unique()
        n_cls_preds = len(unique_labels)
        for x1, y1, x2, y2, obj_id, cls_pred in tracked_objects:
            box_h = int(((y2 - y1) / unpad_h) * img.shape[0])
            box_w = int(((x2 - x1) / unpad_w) * img.shape[1])
            y1 = int(((y1 - pad_y // 2) / unpad_h) * img.shape[0])
            x1 = int(((x1 - pad_x // 2) / unpad_w) * img.shape[1])

            cls = classes[int(cls_pred)]
            if cls == 'person' or cls == 'bicycle': # 0: person, 1: bicycle
                color = colors[int(obj_id) % len(colors)]
                color = [i * 255 for i in color]
                cv2.rectangle(frame, (x1, y1), (x1+box_w, y1+box_h), color, 4)
                cv2.rectangle(frame, (x1, y1-35), (x1+len(cls)*19+60, y1), color, -1)
                cv2.putText(frame, cls + "-" + str(obj_id), (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 3)
                # vivian added
                all_pedestrians.append(Pedestrian((x1, y1), (x1+box_w, y1+box_h), obj_id, cls, box_h, box_w))
    
    #### plotting and showing social distancing violations in video frames ####
    # social distancing 
#     num_violated = calc_num_violated (all_pedestrians)
#     cv2.putText(frame, "Number of People: "+str(len(all_pedestrians))+"; Violations: "+str(num_violated), (10,frame.shape[0]-5), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 3)
    
    # show 2D transformed world coordinates
    warped_frame = cv2.warpPerspective(frame, h, dsize=(width,height))
    warped_frame *= 0 # make all black 
    for p in all_pedestrians:
        color = colors[int(p.id) % len(colors)]
        color = [i * 255 for i in color]
        draw_circle(warped_frame, p.center, p.radius, color)
        # overlap red color if violated 
#         if p.is_violating: 
#             color = (255,0,0)
#             cv2.rectangle(frame, (p.x1, p.y1), (p.x2, p.y2), color, 4)
#             draw_circle(warped_frame, p.center, p.radius, color)
        # append to raw data to store in data frame 
        raw_data_list.append([ii, p.id, p.center[0], p.center[1]])  
        
    ####### show videos in jupyter notebook inline #######
#     fig1=figure(figsize=(12, 8))
#     title("Video Stream")
#     imshow(frame)
    
#     fig2=figure(figsize=(12,8))
#     title("World Coordinate")
#     imshow(warped_frame)
    #######save video to results folder###############
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    result1.write(frame)
    warped_frame = cv2.cvtColor(warped_frame, cv2.COLOR_RGB2BGR)
    result2.write(warped_frame)
    #################################################
#     show()
#     clear_output(wait=True)

In [ ]:
vid.release()
result1.release()
result2.release()

### eyeballing for homography matrix
4 palm trees on google map are: (37.428570, -122.175184), (37.428489, -122.175211), 
                                (37.428589, -122.175260), (37.428506, -122.175288)    
on graph they are approximately (1090, 1575), (2270, 1725), (470, 2050),(2010, 2150)                                 

In [ ]:
vid = cv2.VideoCapture(videopath)
ret, frame = vid.read()
frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

figure, axes = plt.subplots(1,2 )
axes[0].imshow(frame)
axes[0].grid(which='both')

from matplotlib.ticker import AutoMinorLocator
axes[0].xaxis.set_minor_locator(AutoMinorLocator())
axes[0].yaxis.set_minor_locator(AutoMinorLocator())

image_pts = [(1090, 1575), (2270, 1725), (470, 2050),(2010, 2150) ]
for x,y in image_pts: 
    axes[0].scatter(x,y)

# world coord
world_pts = [(37.428570, -122.175184), (37.428489, -122.175211), 
             (37.428589, -122.175260), (37.428506, -122.175288)]
for lat,lon in world_pts: 
    utm_east, utm_north, utm_zone, utm_letter = utm.from_latlon(lat,lon) # meters
    axes[1].scatter(utm_east, utm_north) 
    
axes[1].axis('equal')

In [ ]:
frame

In [ ]:
plt.imshow(cv2.warpPerspective(frame, h, dsize=(1500,1500)))

In [ ]:
h

In [ ]:
np.array(pts_world)-np.array(pts_world).min()

In [ ]:
h

In [ ]:
frame

In [ ]:
cv2.warpPerspective(frame, h, dsize=(1500,1500)).shape